<h1 align="center">Download and load TER data daily</h1>

### Install and import libraries

In [ ]:
pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=af321910692a296f9d52494c8a437372574bb6083e3e7df5f3efe7e44c245371
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
import pandas as pd
import wget
import os
from zipfile import ZipFile
import glob

### Import data

In [ ]:
list_file = glob.glob('*.txt')
for file in list_file:
    os.remove(file)

In [ ]:
url='https://eu.ftp.opendatasoft.com/sncf/gtfs/export-ter-gtfs-last.zip'
filename = "export-ter-gtfs-last.zip"
wget.download(url,out = filename)

zf = ZipFile(f"/content/{filename}")
zf.extractall(path = '/content')
os.remove(filename)

### Clean our dataframes

In [ ]:
routes = pd.read_csv("routes.txt")
trips = pd.read_csv("trips.txt")

In [ ]:
df_trips = pd.merge(routes,trips,on="route_id")
df_trips.dropna(axis=1, inplace=True)
df_trips.drop(["service_id", "trip_headsign", "agency_id", "route_short_name"], axis=1, inplace=True)
df_trips.route_type.replace({0:"light_train", 2:"Train", 3:"Car"},inplace=True)
df_trips.head()

,route_id,route_long_name,route_type,trip_id
0,OCESN-1,Rennes - Brest,Train,OCESN855806F705982:2022-03-22T20:30:31Z
1,OCESN-1,Rennes - Brest,Train,OCESN855807F711795:2022-03-17T20:46:48Z
2,OCESN-1,Rennes - Brest,Train,OCESN855807F711796:2022-03-17T20:46:48Z
3,OCESN-1,Rennes - Brest,Train,OCESN855810F716658:2022-03-16T20:33:02Z
4,OCESN-1,Rennes - Brest,Train,OCESN855814F716659:2022-03-17T20:46:48Z


In [ ]:
df_trips.to_csv("trips.csv")

In [ ]:
stop_times = pd.read_csv("stop_times.txt")
stops = pd.read_csv("stops.txt")

In [ ]:
df_stops = pd.merge(stop_times,stops,on="stop_id")
df_stops.dropna(axis=1, inplace=True)
df_stops.drop(["stop_id", "pickup_type", "drop_off_type", "location_type", "parent_station", "arrival_time"], axis=1, inplace=True)
df_stops.head()

,trip_id,departure_time,stop_sequence,stop_name,stop_lat,stop_lon
0,OCESN105330F714606:2022-03-17T20:46:48Z,07:27:00,0,Saint-Pierre-des-Corps,47.38614,0.723539
1,OCESN105340F711774:2022-03-22T20:30:31Z,12:25:00,0,Saint-Pierre-des-Corps,47.38614,0.723539
2,OCESN105342F711555:2022-03-17T20:46:48Z,17:20:00,0,Saint-Pierre-des-Corps,47.38614,0.723539
3,OCESN105345F812683:2022-03-16T20:33:02Z,19:40:00,0,Saint-Pierre-des-Corps,47.38614,0.723539
4,OCESN105347F774396:2022-03-22T20:30:31Z,22:10:00,0,Saint-Pierre-des-Corps,47.38614,0.723539


In [ ]:
df_stops.to_csv("stops.csv")

In [ ]:
df_station = pd.read_csv("liste-des-gares.csv", sep=";")
df_station.head(1)

,code_uic,libelle,fret,voyageurs,code_ligne,rg_troncon,pk,commune,departemen,idreseau,idgaia,x_l93,y_l93,x_wgs84,y_wgs84,c_geo,geo_point_2d,geo_shape
0,87381715,Mareil-sur-Mauldre,N,O,396000,1,042+382,MAREIL-SUR-MAULDRE,YVELINES,5359,e1b2762a-19bf-11e5-a6ff-01fc64e0362d,617263.3343,6.866554e+06,1.871553,48.893671,"48.8936706358685,1.8715528437305458","48.8936706358,1.87155284373","{""coordinates"": [1.871552843730545, 48.8936706..."


In [ ]:
fret = df_station.fret.replace({"N":"","O":"fret"})
trav = df_station.voyageurs.replace({"N":"","O":"trav"})
station_type = fret + trav
station_type.replace({"frettrav":"both"}, inplace=True)
df_station["station_type"] = station_type
df_station.head(2)

,code_uic,libelle,fret,voyageurs,code_ligne,rg_troncon,pk,commune,departemen,idreseau,idgaia,x_l93,y_l93,x_wgs84,y_wgs84,c_geo,geo_point_2d,geo_shape,station_type
0,87381715,Mareil-sur-Mauldre,N,O,396000,1,042+382,MAREIL-SUR-MAULDRE,YVELINES,5359,e1b2762a-19bf-11e5-a6ff-01fc64e0362d,617263.3343,6.866554e+06,1.871553,48.893671,"48.8936706358685,1.8715528437305458","48.8936706358,1.87155284373","{""coordinates"": [1.871552843730545, 48.8936706...",trav
1,87286062,Hellemmes,O,O,269000,1,004+096,LILLE,NORD,4429,d9b659f8-6667-11e3-89ff-01f464e0362d,708240.9878,7.058436e+06,3.116260,50.622874,"50.62287383634239,3.116260107935155","50.6228738363,3.11626010794","{""coordinates"": [3.116260107935154, 50.6228738...",both


In [ ]:
df_station.to_csv("df_station.csv")